In [18]:
import os, time
from influxdb_client import InfluxDBClient, Point
import random
from kafka import KafkaProducer, KafkaConsumer
import json
import pandas as pd
import json
import datetime as dt
from time import sleep
from confluent_kafka import Producer

In [19]:
token = "pNa6-UfsFJUYd--ryyzXGDUk1o7Ei9BR1TKqO43WJ50uCmuJ30WDAHXk47Nc6vK8wALkJWejAJg09f-0nOE3cg=="
org = "Devnullx"
host = "http://165.232.189.85:8086/"
bucket = "trade_sim2" 
client = InfluxDBClient(url=host, token=token, org=org)

In [20]:
query = '''from(bucket: "trade_sim2")
  |> range(start: -1m)
  |> filter(fn: (r) => r["_measurement"] == "trade")
  |> filter(fn: (r) => r["_field"] == "Close" or r["_field"] == "High" or r["_field"] == "Low" or r["_field"] == "Open" or r["_field"] == "Volume" or r["_field"] == "buy_money" or r["_field"] == "buy_quant" or r["_field"] == "money_earned_user" or r["_field"] == "money_earned_platform" or r["_field"] == "open_time" or r["_field"] == "sell_money" or r["_field"] == "sell_quant" or r["_field"] == "total_money_invested" or r["_field"] == "user_loser" or r["_field"] == "user_profitable")
  |> aggregateWindow(every: 1m, fn: mean, createEmpty: false)
  |> yield(name: "mean")'''

In [21]:
query_api = client.query_api()

In [ ]:
while True:
    tables = query_api.query(query=query, org=org)
    records = []
    for table in tables:
        for record in table.records:
            records.append({
                'time': record.get_time(),
                '_value': record.get_value(),
                '_field': record.get_field(),
                '_measurement': record.get_measurement(),
                'tags': record.values  # Extract tag values if needed
            })

    df = pd.DataFrame(records)
    df = df.sort_values(by="time")

    df = df.drop(["time"],axis = 1)
    producer = KafkaProducer(bootstrap_servers=['139.59.1.5:9092'])
    key = str(0).encode()  # Here, '0' is just an example; it can be any unique identifier
    df_dict = df.to_dict()
    data = json.dumps(df_dict, default=str).encode('utf-8')
    producer.send(topic="test-topic", key=key, value=data)
    print(f'Sent record to topic at time {dt.datetime.utcnow()}')
    sleep(60)

Sent record to topic at time 2024-09-18 18:14:12.508016
Sent record to topic at time 2024-09-18 18:15:12.677064
Sent record to topic at time 2024-09-18 18:16:12.870283
Sent record to topic at time 2024-09-18 18:17:13.063297
Sent record to topic at time 2024-09-18 18:18:13.255273
Sent record to topic at time 2024-09-18 18:19:13.436648
